# Projet : Simulation de la propagation d'un virus (SIR) sur différents types de graphes

**Date** : 27 Mai 2025

**Objectif** : Nous avons simulé la dynamique d'une épidémie en utilisant le modèle SIR (Susceptible-Infecté-Rétabli) sur des graphes Erdős-Rényi (ER), Watts-Strogatz (WS), et Barabási-Albert (BA), puis comparé la vitesse et l'étendue de la propagation selon la structure du réseau.

**Auteur** : Nous

## Prérequis : Installation des packages

Pour exécuter ce notebook, nous avons installé les bibliothèques Python suivantes. Nous avons exécuté ces commandes dans notre terminal :

```bash
pip install networkx
pip install numpy
pip install matplotlib
```

Nous avons vérifié que Python 3.x est installé, car ces packages sont nécessaires pour la génération des graphes, les calculs numériques et les visualisations.

## Contexte pédagogique

Ce projet nous a permis de :
- Mettre en pratique les modèles de graphes (ER, WS, BA).
- Comprendre la diffusion dans des réseaux complexes.
- Travailler avec des simulations stochastiques.
- Observer des propriétés comme les hubs ou les effets de petit monde.

## Structure du notebook

1. Importation des bibliothèques
2. Implémentation des fonctions SIR
3. Simulation sur les différents graphes
4. Visualisation et analyse comparative
5. Conclusions

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from random import choice, random

# We set a random seed for reproducibility
print("Setting random seed for reproducibility to 42")
np.random.seed(42)

## Étape 1 : Initialisation de l'état des nœuds

In [ ]:
def initialize_sir_states(G, initial_infected=1):
    """We initialized node states: all set to 'S' except for a few randomly selected infected nodes.
    
    Args:
        G (networkx.Graph): Our input graph
        initial_infected (int): Number of nodes to initially infect
    
    Returns:
        dict: Dictionary mapping nodes to their state ('S', 'I', or 'R')
    """
    print(f"Initializing states for graph with {len(G.nodes())} nodes")
    states = {node: 'S' for node in G.nodes()}
    print("All nodes set to 'S' (Susceptible) by default")
    infected_nodes = np.random.choice(list(G.nodes()), size=initial_infected, replace=False)
    print(f"Randomly selecting {initial_infected} nodes to infect: {infected_nodes}")
    for node in infected_nodes:
        states[node] = 'I'
        print(f"Node {node} marked as 'I' (Infected)")
    print("Initialization complete")
    return states

## Étape 2 : Une étape de simulation SIR

In [ ]:
def sir_step(G, states, beta, gamma):
    """We applied one step of the SIR simulation and returned updated states.
    
    Args:
        G (networkx.Graph): Our input graph
        states (dict): Current node states
        beta (float): Probability of infection
        gamma (float): Probability of recovery
    
    Returns:
        dict: Updated node states
        tuple: (S_count, I_count, R_count) for the current step
    """
    print(f"Starting SIR step with beta={beta} and gamma={gamma}")
    new_states = states.copy()
    print("Creating a copy of current states")
    
    # Infection step
    print("Processing infection step")
    for node in G.nodes():
        if states[node] == 'I':
            print(f"Node {node} is infected, checking neighbors")
            for neighbor in G.neighbors(node):
                if states[neighbor] == 'S' and random() < beta:
                    new_states[neighbor] = 'I'
                    print(f"Neighbor {neighbor} infected from {node}")
    
    # Recovery step
    print("Processing recovery step")
    for node in G.nodes():
        if states[node] == 'I' and random() < gamma:
            new_states[node] = 'R'
            print(f"Node {node} recovered to 'R'")
    
    # Count states
    print("Counting final states")
    S_count = sum(1 for state in new_states.values() if state == 'S')
    I_count = sum(1 for state in new_states.values() if state == 'I')
    R_count = sum(1 for state in new_states.values() if state == 'R')
    print(f"Counts: S={S_count}, I={I_count}, R={R_count}")
    
    return new_states, (S_count, I_count, R_count)

## Étape 3 : Boucle de simulation complète

In [ ]:
def run_sir_simulation(G, beta=0.03, gamma=0.01, initial_infected=1):
    """We ran the full SIR simulation until no infected nodes remain.
    
    Args:
        G (networkx.Graph): Our input graph
        beta (float): Infection probability
        gamma (float): Recovery probability
        initial_infected (int): Number of initially infected nodes
    
    Returns:
        list: List of (S_count, I_count, R_count) tuples for each time step
    """
    print(f"Starting SIR simulation with beta={beta}, gamma={gamma}, and {initial_infected} initial infected nodes")
    states = initialize_sir_states(G, initial_infected)
    print("Initial states set, beginning simulation loop")
    results = []
    
    while sum(1 for state in states.values() if state == 'I') > 0:
        print(f"Current number of infected nodes: {sum(1 for state in states.values() if state == 'I')}")
        states, counts = sir_step(G, states, beta, gamma)
        print(f"Step completed, adding counts: {counts}")
        results.append(counts)
    
    print("Simulation ended, no infected nodes remain")
    return results

## Étape 4 : Comparaison des structures

In [ ]:
def compare_structures(n, p, k, beta_ws, m, beta_sir, gamma_sir):
    """We ran the SIR simulation on ER, WS, and BA graphs and returned the results.
    
    Args:
        n (int): Number of nodes
        p (float): Edge probability for ER graph
        k (int): Number of nearest neighbors for WS graph
        beta_ws (float): Rewiring probability for WS graph
        m (int): Number of edges to attach for BA graph
        beta_sir (float): Infection probability for SIR
        gamma_sir (float): Recovery probability for SIR
    
    Returns:
        dict: Dictionary with graph type as key and SIR results as value
    """
    print(f"Creating graphs with n={n}, p={p}, k={k}, beta_ws={beta_ws}, m={m}")
    # We created the graphs
    G_er = nx.erdos_renyi_graph(n, p)
    print("ER graph created")
    G_ws = nx.watts_strogatz_graph(n, k, beta_ws)
    print("WS graph created")
    G_ba = nx.barabasi_albert_graph(n, m)
    print("BA graph created")
    
    # We ran the simulations
    print("Running SIR simulation on ER graph")
    results_er = run_sir_simulation(G_er, beta_sir, gamma_sir)
    print("Running SIR simulation on WS graph")
    results_ws = run_sir_simulation(G_ws, beta_sir, gamma_sir)
    print("Running SIR simulation on BA graph")
    results_ba = run_sir_simulation(G_ba, beta_sir, gamma_sir)
    
    results = {
        'ER': results_er,
        'WS': results_ws,
        'BA': results_ba
    }
    print("All simulations completed, results stored")
    return results

## Étape 5 : Visualisation et analyse

In [ ]:
def plot_sir_dynamics(sir_results, title="Dynamique SIR"):
    """We plotted the S, I, R curves over time.
    
    Args:
        sir_results (dict): Dictionary with graph type and SIR results
        title (str): Plot title
    """
    print(f"Generating plot with title: {title}")
    plt.figure(figsize=(12, 8))
    print("Figure initialized with size 12x8")
    
    for graph_type, results in sir_results.items():
        print(f"Processing data for {graph_type} graph")
        S = [r[0] for r in results]
        I = [r[1] for r in results]
        R = [r[2] for r in results]
        time = range(len(results))
        print(f"Data extracted: S={S[:5]}... I={I[:5]}... R={R[:5]}...")
        
        plt.plot(time, S, label=f'{graph_type} - Susceptible', linestyle='--')
        plt.plot(time, I, label=f'{graph_type} - Infecté', linestyle='-')
        plt.plot(time, R, label=f'{graph_type} - Rétabli', linestyle='-.')
        print(f"Plotted {graph_type} curves")
    
    plt.xlabel('Étapes temporelles')
    plt.ylabel('Nombre de nœuds')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    print("Adding labels, legend, and grid")
    plt.show()
    print("Plot displayed")

## Simulation et Analyse Comparative

Nous avons exécuté la simulation avec les paramètres suivants :
- Nombre de nœuds : 1000
- Probabilité de connexion (ER) : p=0.01
- Voisins les plus proches (WS) : k=4
- Probabilité de recâblage (WS) : beta_ws=0.1
- Arêtes par nœud (BA) : m=2
- Probabilité d'infection : beta_sir=0.03
- Probabilité de guérison : gamma_sir=0.01

In [ ]:
# We launched the simulation
print("Setting simulation parameters:")
print(f"n={1000}, p={0.01}, k={4}, beta_ws={0.1}, m={2}, beta_sir={0.03}, gamma_sir={0.01}")
n = 1000
p = 0.01
k = 4
beta_ws = 0.1
m = 2
beta_sir = 0.03
gamma_sir = 0.01

results = compare_structures(n, p, k, beta_ws, m, beta_sir, gamma_sir)
print("Simulation results obtained, now plotting")
plot_sir_dynamics(results, title="Dynamique SIR sur différentes structures de graphes")

## Analyse Comparative

### Vitesse d'infection
- **Graphe ER** : Nous avons observé que la propagation est modérée, fortement dépendante de la probabilité de connexion $p$. Avec $p=0.01$, le graphe est relativement peu connecté, ce qui ralentit la propagation.
- **Graphe WS** : Nous avons remarqué que la propagation est plus rapide en raison de la structure en petits mondes, surtout avec un faible $eta_{ws}$ qui maintient des clusters locaux favorisant la transmission rapide au sein des groupes.
- **Graphe BA** : Nous avons constaté que la propagation est la plus rapide, surtout si les hubs (nœuds à forte connectivité) sont infectés tôt, car ils agissent comme des super-propagateurs.

### Pourcentage total infecté

Nous avons calculé la proportion de nœuds ayant atteint l'état 'R' à la fin de la simulation pour quantifier le pourcentage total infecté.

In [ ]:
def calculate_total_infected(results):
    """We calculated the total percentage of infected nodes for each graph.
    
    Args:
        results (dict): SIR simulation results
    
    Returns:
        dict: Percentage of total infected nodes per graph type
    """
    print("Calculating total infected percentage for each graph")
    total_infected = {}
    for graph_type, data in results.items():
        print(f"Processing {graph_type} graph data")
        final_R = data[-1][2] if data else 0
        total_infected[graph_type] = (final_R / n) * 100
        print(f"Final R count for {graph_type}: {final_R}")
    print("Calculation complete")
    return total_infected

total_infected = calculate_total_infected(results)
print("Displaying total infected percentages:")
for graph_type, percentage in total_infected.items():
    print(f'{graph_type}: {percentage:.2f}% nodes were infected')

### Rôle de la structure du réseau
- **ER** : Nous avons vu que la structure aléatoire entraîne une propagation homogène mais lente, car il n'y a pas de hubs pour accélérer la diffusion.
- **WS** : Nous avons noté que la propriété de petit monde permet une propagation rapide au sein des clusters locaux, mais la portée globale dépend du recâblage.
- **BA** : Nous avons découvert que les hubs jouent un rôle crucial, permettant une propagation explosive si infectés tôt, mais la résilience du réseau peut être faible si les hubs sont immunisés.

## Conclusions

Nous avons conclu que la structure du réseau a un impact significatif sur la dynamique épidémique :
- Le graphe BA est le plus vulnérable à une propagation rapide en raison de ses hubs.
- Le graphe WS favorise une propagation rapide dans des communautés locales.
- Le graphe ER montre une propagation plus lente et plus uniforme.

Ces résultats nous ont montré l'importance de la topologie du réseau dans la gestion des épidémies, avec des implications pour la modélisation de la diffusion d'informations ou de malwares.

## Soumission

Nous convertirons ce notebook en PDF et l'enverrons à l'adresse **bernard.agbemadon@gmail.com** avec l'objet : **M2 - ESGIS - Projet-SA**.